# Imports

In [1]:
import requests as re
import json
import pandas as pd
import sqlalchemy
import pymysql
from sqlalchemy import create_engine


In [2]:
from datetime import datetime

# Connection to Flights API

In [3]:
response = re.get("http://aviation-edge.com/v2/public/flights?key=8bbf2c-a12972&depIata=BCN")
response.headers

{'Date': 'Wed, 20 Mar 2019 08:17:28 GMT', 'Server': 'Apache', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Headers': 'X-Requested-With, Content-Type, Accept, Origin, Authorization', 'Access-Control-Allow-Methods': 'GET, POST, PUT, DELETE, OPTIONS', 'Cache-Control': 'max-age=172800', 'Expires': 'Fri, 22 Mar 2019 08:17:28 GMT', 'Vary': 'Accept-Encoding,User-Agent', 'Content-Encoding': 'gzip', 'Content-Length': '4475', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive', 'Content-Type': 'text/html; charset=UTF-8'}

In [4]:
flights_api = response.json()
print(json.dumps(flights_api,sort_keys=True, indent=4))

[
    {
        "aircraft": {
            "iataCode": "A320",
            "icao24": "",
            "icaoCode": "A320",
            "regNumber": "OEIZE"
        },
        "airline": {
            "iataCode": "U2",
            "icaoCode": "EZY"
        },
        "arrival": {
            "iataCode": "LGW",
            "icaoCode": "EGKK"
        },
        "departure": {
            "iataCode": "BCN",
            "icaoCode": "LEBL"
        },
        "flight": {
            "iataNumber": "U216EF",
            "icaoNumber": "EZY16EF",
            "number": "16EF"
        },
        "geography": {
            "altitude": 2346.96,
            "direction": 264.23,
            "latitude": 51.0809,
            "longitude": 7.4976
        },
        "speed": {
            "horizontal": 515.628,
            "isGround": 0,
            "vertical": -29.268
        },
        "status": "en-route",
        "system": {
            "squawk": "1105",
            "updated": "1553069759"
        }
    },

# Generate Series with the information we need

In [5]:
plane_iata = pd.Series([plane['aircraft']['iataCode'] for plane in flights_api])
plane_icao = pd.Series([plane['aircraft']['icaoCode'] for plane in flights_api])
dep_iata = pd.Series([plane['departure']['iataCode'] for plane in flights_api])
arr_iata = pd.Series([plane['arrival']['iataCode'] for plane in flights_api])
flight = pd.Series([plane['flight']['number'] for plane in flights_api])
latitude = pd.Series([plane['geography']['latitude'] for plane in flights_api])
longitude = pd.Series([plane['geography']['longitude'] for plane in flights_api])
hor_speed = pd.Series([plane['speed']['horizontal'] for plane in flights_api])
ver_speed = pd.Series([plane['speed']['vertical'] for plane in flights_api])
status = pd.Series([plane['status'] for plane in flights_api])
updated_time = pd.Series([plane['system']['updated'] for plane in flights_api])

# Create DataFrame from the Series

In [6]:
flights = pd.DataFrame({'aircraft_iata': plane_iata, 'aircraft_icao': plane_icao, 'departure': dep_iata, 'arrival': arr_iata, 'flight_number': flight, 'latitude': latitude, 'longitude': longitude, 'horizontal_speed': hor_speed, 'vertical_speed': ver_speed, 'status': status, 'updated_time': updated_time})

In [7]:
flights

,aircraft_iata,aircraft_icao,departure,arrival,flight_number,latitude,longitude,horizontal_speed,vertical_speed,status,updated_time
0,A320,A320,BCN,LGW,16EF,51.08090,7.497600,515.628,-29.26800,en-route,1553069759
1,A320,A320,BCN,MAD,9010,41.29020,2.070340,0.000,0.00000,landed,1553069705
2,A320,A320,BCN,LGW,8570,50.85410,-0.386200,467.136,-8.20800,en-route,1553069699
3,A320,A320,BCN,MAD,7451,40.87310,-0.253747,975.477,0.00000,en-route,1553069641
4,A20N,A20N,BCN,LHR,477,46.49340,-1.002770,758.910,0.00000,en-route,1553069641
5,A320,A320,BCN,BRU,8988,46.44590,0.794800,740.160,0.00000,en-route,1553069640
6,B738,B738,BCN,SXF,149,51.41870,13.550700,637.200,-37.44000,en-route,1553069640
7,A320,A320,BCN,ORY,80XK,45.59270,1.246100,722.052,0.00000,started,1553069640
8,A321,A321,BCN,LGW,782Y,49.66030,-1.689200,750.276,0.00000,en-route,1553069639
9,A320,A320,BCN,ALC,1312,40.94110,1.503700,779.400,38.62800,en-route,1553069579


# Converting timestamps into datetimes

In [8]:
flights['updated_time'] = flights['updated_time'].apply(lambda x: datetime.utcfromtimestamp(int(x)).strftime('%Y-%m-%d %H:%M:%S'))

# Convert speeds values into floats, and adding total speed (pitagoras)

In [9]:
flights['horizontal_speed'] = flights['horizontal_speed'].apply(float)
flights['vertical_speed'] = flights['vertical_speed'].apply(float)
flights['total_speed'] = round((flights['horizontal_speed']**2 + flights['vertical_speed']**2)**0.5,2)

# Checkout there are not null values

In [10]:
flights.isnull().sum()

aircraft_iata       0
aircraft_icao       0
departure           0
arrival             0
flight_number       0
latitude            0
longitude           0
horizontal_speed    0
vertical_speed      0
status              0
updated_time        0
total_speed         0
dtype: int64

# Export to Database

In [11]:
driver = 'mysql+pymysql'
ip = '35.246.218.28'
username = 'root'
password = 'root'
db = 'flight_radar'

cs  = f'{driver}://{username}:{password}@{ip}/{db}'
engine = create_engine(cs)

In [12]:
flights.to_sql(con=engine, name='flights_20_3_9:00', if_exists='replace')